In [0]:
%sql
create schema gold

In [0]:
%sql
use schema silver

In [0]:
%sql
WITH CityStateTotals AS (
    SELECT
        a.State,
        a.City,
        COUNT(DISTINCT o.CustomerID) AS TotalCustomers,
        COUNT(DISTINCT o.OrderID) AS TotalOrders,
        COUNT(oi.ProductID) AS TotalProductsSold,
        SUM(CASE WHEN r.OrderID IS NOT NULL THEN oi.Quantity ELSE 0 END) AS TotalProductsReturned,
        COUNT(DISTINCT r.OrderID) * 100.0 / COUNT(DISTINCT o.OrderID) AS ReturnRate,
        SUM(oi.Quantity * p.Discounted_Price) AS TotalRevenue,
        AVG(DATEDIFF(o.ActualDeliveryDate, o.ShippingDate)) AS AvgDeliveryTime
    FROM
        orders o
        JOIN order_items oi ON o.OrderID = oi.OrderID
        LEFT JOIN returns r ON o.OrderID = r.OrderID
        JOIN products p ON oi.ProductID = p.Product_ID
        JOIN addresses a ON o.CustomerID = a.CustomerID
    GROUP BY
        a.State, a.City
),
TopVendorByCityState AS (
    SELECT
        a.State,
        a.City,
        v.VendorName,
        SUM(oi.Quantity * p.Discounted_Price) AS VendorRevenue,
        RANK() OVER (PARTITION BY a.State, a.City ORDER BY SUM(oi.Quantity * p.Discounted_Price) DESC) AS Rank
    FROM
        orders o
        JOIN order_items oi ON o.OrderID = oi.OrderID
        JOIN products p ON oi.ProductID = p.Product_ID
        JOIN vendors v ON o.VendorID = v.VendorID
        JOIN addresses a ON o.CustomerID = a.CustomerID
    GROUP BY
        a.State, a.City, v.VendorName
)
SELECT
    cst.State,
    cst.City,
    cst.TotalCustomers,
    cst.TotalOrders,
    cst.TotalProductsSold AS TotalProductsSold,
    cst.TotalProductsReturned,
    ROUND(cst.ReturnRate, 2) AS ReturnRate,
    cst.TotalRevenue,
    tvb.VendorName AS TopVendorByRevenue,
    ROUND(cst.AvgDeliveryTime, 2) AS AvgDeliveryTime
FROM
    CityStateTotals cst
    LEFT JOIN TopVendorByCityState tvb ON cst.State = tvb.State AND cst.City = tvb.City AND tvb.Rank = 1
ORDER BY
    cst.State, cst.City;


In [0]:

from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Load tables
orders = spark.table("orders")
order_items = spark.table("order_items")
returns = spark.table("returns")
products = spark.table("products")
addresses = spark.table("addresses")
vendors = spark.table("vendors")

# Compute CityStateTotals
city_state_totals = (
    orders
    .join(order_items, "OrderID")
    .join(products, order_items["ProductID"] == products["Product_ID"])
    .join(addresses, "CustomerID")
    .join(returns, "OrderID", "left")
    .groupBy("State", "City")
    .agg(
        F.countDistinct("orders.CustomerID").alias("TotalCustomers"),
        F.countDistinct("orders.OrderID").alias("TotalOrders"),
        F.count("order_items.ProductID").alias("TotalProductsSold"),
        F.sum(F.when(returns["OrderID"].isNotNull(), order_items["Quantity"]).otherwise(0)).alias("TotalProductsReturned"),
        (F.countDistinct("returns.OrderID") * 100.0 / F.countDistinct("orders.OrderID")).alias("ReturnRate"),
        F.sum(order_items["Quantity"] * products["Discounted_Price"]).alias("TotalRevenue"),
        F.avg(F.datediff("orders.ActualDeliveryDate", "orders.ShippingDate")).alias("AvgDeliveryTime")
    )
)

# Compute TopVendorByCityState
window_spec = Window.partitionBy("State", "City").orderBy(F.col("VendorRevenue").desc())
top_vendor_by_city_state = (
    orders
    .join(order_items, "OrderID")
    .join(products, order_items["ProductID"] == products["Product_ID"])
    .join(vendors, "VendorID")
    .join(addresses, "CustomerID")
    .groupBy("State", "City", "VendorName")
    .agg(F.sum(order_items["Quantity"] * products["Discounted_Price"]).alias("VendorRevenue"))
    .withColumn("Rank", F.rank().over(window_spec))
    .filter(F.col("Rank") == 1)
)

# Join results
final_result = (
    city_state_totals
    .join(top_vendor_by_city_state, ["State", "City"], "left")
    .select(
        "State",
        "City",
        "TotalCustomers",
        "TotalOrders",
        "TotalProductsSold",
        "TotalProductsReturned",
        F.round("ReturnRate", 2).alias("ReturnRate"),
        "TotalRevenue",
        F.col("VendorName").alias("TopVendorByRevenue"),
        F.round("AvgDeliveryTime", 2).alias("AvgDeliveryTime")
    )
    .orderBy("State", "City")
)

# Display the final result
display(final_result)

# Save as Delta table
final_result.write.format("delta").mode("overwrite").saveAsTable("gold.Regional_analysis")

In [0]:
%sql
select * from gold.Regional_analysis

In [0]:
%sql
WITH PaymentTotals AS (
    SELECT
        pm.PaymentMethodID,
        pm.MethodName,
        COUNT(DISTINCT p.OrderID) AS TotalOrdersPaid,
        SUM(p.CouponAmount + p.GiftCardAmount) AS TotalPaymentAmount,
        SUM(p.CouponAmount) AS TotalCouponAmount,
        SUM(p.GiftCardAmount) AS TotalGiftCardAmount
    FROM
        payment_methods pm
        JOIN payments p ON pm.PaymentMethodID = p.PaymentMethodID
    GROUP BY
        pm.PaymentMethodID, pm.MethodName
),
CouponUsage AS (
    SELECT
        pm.PaymentMethodID,
        COUNT(CASE WHEN p.CouponUsage = 'Yes' THEN 1 ELSE NULL END) AS TotalCouponUsage
    FROM
        payment_methods pm
        JOIN payments p ON pm.PaymentMethodID = p.PaymentMethodID
    GROUP BY
        pm.PaymentMethodID
),
GiftCardUsage AS (
    SELECT
        pm.PaymentMethodID,
        COUNT(CASE WHEN p.GiftCardUsage = 'Yes' THEN 1 ELSE NULL END) AS TotalGiftCardUsage
    FROM
        payment_methods pm
        JOIN payments p ON pm.PaymentMethodID = p.PaymentMethodID
    GROUP BY
        pm.PaymentMethodID
),
PaymentAnalysis AS (
    SELECT
        pt.PaymentMethodID,
        pt.MethodName,
        pt.TotalOrdersPaid,
        pt.TotalPaymentAmount,
        pt.TotalCouponAmount,
        pt.TotalGiftCardAmount,
        cu.TotalCouponUsage,
        gcu.TotalGiftCardUsage
    FROM
        PaymentTotals pt
        LEFT JOIN CouponUsage cu ON pt.PaymentMethodID = cu.PaymentMethodID
        LEFT JOIN GiftCardUsage gcu ON pt.PaymentMethodID = gcu.PaymentMethodID
)
SELECT
    PaymentMethodID,
    MethodName,
    TotalOrdersPaid,
    TotalPaymentAmount,
    COALESCE(TotalCouponUsage, 0) AS TotalCouponUsed,
    TotalCouponAmount,
    COALESCE(TotalGiftCardUsage, 0) AS TotalGiftCardUsed,
    TotalGiftCardAmount
FROM
    PaymentAnalysis
ORDER BY
    TotalOrdersPaid DESC;


In [0]:
# Load tables
payment_methods = spark.table("payment_methods")
payments = spark.table("payments")

# Compute PaymentTotals
payment_totals = (
    payment_methods
    .join(payments, "PaymentMethodID")
    .groupBy("PaymentMethodID", "MethodName")
    .agg(
        F.countDistinct("OrderID").alias("TotalOrdersPaid"),
        F.sum(F.col("CouponAmount") + F.col("GiftCardAmount")).alias("TotalPaymentAmount"),
        F.sum("CouponAmount").alias("TotalCouponAmount"),
        F.sum("GiftCardAmount").alias("TotalGiftCardAmount")
    )
)

# Compute CouponUsage
coupon_usage = (
    payment_methods
    .join(payments, "PaymentMethodID")
    .groupBy("PaymentMethodID")
    .agg(
        F.count(F.when(F.col("CouponUsage") == 'Yes', 1)).alias("TotalCouponUsage")
    )
)

# Compute GiftCardUsage
gift_card_usage = (
    payment_methods
    .join(payments, "PaymentMethodID")
    .groupBy("PaymentMethodID")
    .agg(
        F.count(F.when(F.col("GiftCardUsage") == 'Yes', 1)).alias("TotalGiftCardUsage")
    )
)

# Compute PaymentAnalysis
payment_analysis = (
    payment_totals
    .join(coupon_usage, "PaymentMethodID", "left")
    .join(gift_card_usage, "PaymentMethodID", "left")
    .select(
        "PaymentMethodID",
        "MethodName",
        "TotalOrdersPaid",
        "TotalPaymentAmount",
        "TotalCouponAmount",
        "TotalGiftCardAmount",
        F.coalesce("TotalCouponUsage", F.lit(0)).alias("TotalCouponUsed"),
        F.coalesce("TotalGiftCardUsage", F.lit(0)).alias("TotalGiftCardUsed")
    )
    .orderBy(F.col("TotalOrdersPaid").desc())
)

# Display the final result
display(payment_analysis)

# Save as Delta table
payment_analysis.write.format("delta").mode("overwrite").saveAsTable("gold.payment_analysis")

In [0]:
%sql
select * from gold.payment_analysis

In [0]:
%sql
WITH OrderTotals AS (
    SELECT
        o.CustomerID,
        COUNT(DISTINCT o.OrderID) AS NumOrders,
        SUM(oi.Quantity) AS BasketSize,
        MIN(o.OrderDate) AS FirstOrderDate,
        MAX(o.OrderDate) AS LastOrderDate,
        SUM(oi.Quantity * p.Discounted_Price) AS TotalPurchaseValue
    FROM
        orders o
        JOIN order_items oi ON o.OrderID = oi.OrderID
        JOIN products p ON oi.ProductID = p.Product_ID
    GROUP BY
        o.CustomerID
),
CouponGiftCardUsage AS (
    SELECT
        o.CustomerID,
        COUNT(CASE WHEN p.CouponUsage = 'Yes' THEN 1 ELSE NULL END) AS TotalCouponUsage,
        SUM(p.CouponAmount) AS TotalCouponAmount,
        COUNT(CASE WHEN p.GiftCardUsage = 'Yes' THEN 1 ELSE NULL END) AS TotalGiftCardUsage,
        SUM(p.GiftCardAmount) AS TotalGiftCardAmount
    FROM
        payments p
        JOIN orders o ON p.OrderID = o.OrderID
    GROUP BY
        o.CustomerID
),
CustomerAnalysis AS (
    SELECT
        ot.CustomerID,
        c.Email,
        ot.NumOrders,
        ROUND(ot.BasketSize / ot.NumOrders, 0) AS AvgBasketSize,
        ROUND(DATEDIFF(ot.LastOrderDate, ot.FirstOrderDate) / NULLIF(ot.NumOrders - 1, 0), 0) AS PurchaseFrequency,
        ot.TotalPurchaseValue,
        ROUND(ot.TotalPurchaseValue / ot.NumOrders, 2) AS AvgBasketValue,
        COALESCE(cgu.TotalCouponUsage, 0) AS TotalCouponUsed,
        COALESCE(cgu.TotalCouponAmount, 0) AS TotalCouponAmount,
        COALESCE(cgu.TotalGiftCardUsage, 0) AS TotalGiftCardUsed,
        COALESCE(cgu.TotalGiftCardAmount, 0) AS TotalGiftCardAmount
    FROM
        OrderTotals ot
        JOIN customers c ON ot.CustomerID = c.CustomerID
        LEFT JOIN CouponGiftCardUsage cgu ON ot.CustomerID = cgu.CustomerID
)
SELECT
    CustomerID,
    Email,
    NumOrders,
    AvgBasketSize,
    PurchaseFrequency,
    TotalPurchaseValue,
    AvgBasketValue,
    TotalCouponUsed,
    TotalCouponAmount,
    TotalGiftCardUsed,
    TotalGiftCardAmount
FROM
    CustomerAnalysis
ORDER BY
    NumOrders DESC;


In [0]:
# Load tables
orders = spark.table("orders")
order_items = spark.table("order_items")
products = spark.table("products")
payments = spark.table("payments")
customers = spark.table("customers")

# Compute OrderTotals
order_totals = (
    orders
    .join(order_items, "OrderID")
    .join(products, order_items["ProductID"] == products["Product_ID"])
    .groupBy("CustomerID")
    .agg(
        F.countDistinct("orders.OrderID").alias("NumOrders"),
        F.sum(order_items["Quantity"]).alias("BasketSize"),
        F.min("orders.OrderDate").alias("FirstOrderDate"),
        F.max("orders.OrderDate").alias("LastOrderDate"),
        F.sum(order_items["Quantity"] * products["Discounted_Price"]).alias("TotalPurchaseValue")
    )
)

# Compute CouponGiftCardUsage
coupon_gift_card_usage = (
    payments
    .join(orders, "OrderID")
    .groupBy("CustomerID")
    .agg(
        F.count(F.when(F.col("CouponUsage") == 'Yes', 1)).alias("TotalCouponUsage"),
        F.sum("CouponAmount").alias("TotalCouponAmount"),
        F.count(F.when(F.col("GiftCardUsage") == 'Yes', 1)).alias("TotalGiftCardUsage"),
        F.sum("GiftCardAmount").alias("TotalGiftCardAmount")
    )
)

# Compute CustomerAnalysis
customer_analysis = (
    order_totals
    .join(customers, "CustomerID")
    .join(coupon_gift_card_usage, "CustomerID", "left")
    .select(
        "CustomerID",
        "Email",
        "NumOrders",
        F.round(F.col("BasketSize") / F.col("NumOrders"), 0).alias("AvgBasketSize"),
        F.round(F.datediff("LastOrderDate", "FirstOrderDate") / F.when(F.col("NumOrders") - 1 != 0, F.col("NumOrders") - 1).otherwise(1), 0).alias("PurchaseFrequency"),
        "TotalPurchaseValue",
        F.round(F.col("TotalPurchaseValue") / F.col("NumOrders"), 2).alias("AvgBasketValue"),
        F.coalesce("TotalCouponUsage", F.lit(0)).alias("TotalCouponUsed"),
        F.coalesce("TotalCouponAmount", F.lit(0)).alias("TotalCouponAmount"),
        F.coalesce("TotalGiftCardUsage", F.lit(0)).alias("TotalGiftCardUsed"),
        F.coalesce("TotalGiftCardAmount", F.lit(0)).alias("TotalGiftCardAmount")
    )
    .orderBy(F.col("NumOrders").desc())
)

# Display the final result
display(customer_analysis)

#save to table
customer_analysis.write.format("delta").mode("overwrite").saveAsTable("gold.customer_analysis")

In [0]:
%sql
select * from gold.customer_analysis

In [0]:
%sql
WITH ProductTotals AS (
    SELECT
        p.Product_ID,
        p.Product_Name,
        COUNT(DISTINCT(oi.OrderID)) AS TotalOrders,
        SUM(oi.Quantity) AS TotalQuantitySold,
        SUM(oi.Quantity * p.Discounted_Price) AS TotalRevenue,
        AVG(p.Actual_Price) AS AvgPrice
    FROM
        products p
        JOIN order_items oi ON p.Product_ID = oi.ProductID
        JOIN orders o ON oi.OrderID = o.OrderID
    GROUP BY
        p.Product_ID, p.Product_Name
),
ReturnStats AS (
    SELECT
        p.Product_ID,
        COUNT(r.OrderID) AS TotalReturns
    FROM
        products p
        JOIN order_items oi ON p.Product_ID = oi.ProductID
        JOIN returns r ON oi.OrderID = r.OrderID
    GROUP BY
        p.Product_ID
),
ProductAnalysis AS (
    SELECT
        pt.Product_ID,
        pt.Product_Name,
        pt.TotalOrders,
        pt.TotalQuantitySold,
        pt.TotalRevenue,
        pt.AvgPrice,
        rs.TotalReturns,
        ROUND((rs.TotalReturns / NULLIF(pt.TotalOrders, 0)) * 100, 2) AS ReturnRate
    FROM
        ProductTotals pt
        LEFT JOIN ReturnStats rs ON pt.Product_ID = rs.Product_ID
)
SELECT
    Product_ID,
    Product_Name,
    TotalOrders,
    TotalQuantitySold,
    TotalRevenue,
    AvgPrice,
    COALESCE(TotalReturns, 0) AS TotalReturns,
    COALESCE(ReturnRate, 0) AS ReturnRate
FROM
    ProductAnalysis
ORDER BY
    TotalRevenue DESC;


In [0]:
# Load tables
products = spark.table("products")
order_items = spark.table("order_items")
orders = spark.table("orders")
returns = spark.table("returns")

# Compute ProductTotals
product_totals = (
    products
    .join(order_items, products["Product_ID"] == order_items["ProductID"])
    .join(orders, order_items["OrderID"] == orders["OrderID"])
    .groupBy(products["Product_ID"], products["Product_Name"])
    .agg(
        F.countDistinct(order_items["OrderID"]).alias("TotalOrders"),
        F.sum(order_items["Quantity"]).alias("TotalQuantitySold"),
        F.sum(order_items["Quantity"] * products["Discounted_Price"]).alias("TotalRevenue"),
        F.avg(products["Actual_Price"]).alias("AvgPrice")
    )
)

# Compute ReturnStats
return_stats = (
    products
    .join(order_items, products["Product_ID"] == order_items["ProductID"])
    .join(returns, order_items["OrderID"] == returns["OrderID"])
    .groupBy(products["Product_ID"])
    .agg(
        F.count(returns["OrderID"]).alias("TotalReturns")
    )
)

# Compute ProductAnalysis
product_analysis = (
    product_totals
    .join(return_stats, "Product_ID", "left")
    .select(
        "Product_ID",
        "Product_Name",
        "TotalOrders",
        "TotalQuantitySold",
        "TotalRevenue",
        "AvgPrice",
        F.coalesce(return_stats["TotalReturns"], F.lit(0)).alias("TotalReturns"),
        F.round((F.coalesce(return_stats["TotalReturns"], F.lit(0)) / F.when(product_totals["TotalOrders"] != 0, product_totals["TotalOrders"]).otherwise(1)) * 100, 2).alias("ReturnRate")
    )
    .orderBy(F.col("TotalRevenue").desc())
)

# Display the final result
display(product_analysis)

#save to table
product_analysis.write.format("delta").mode("overwrite").saveAsTable("gold.product_analysis")

In [0]:
%sql
select * from gold.product_analysis

In [0]:
%sql
WITH VendorTotals AS (
    SELECT
        v.VendorID,
        v.VendorName,
        COUNT(DISTINCT o.OrderID) AS TotalOrders,
        SUM(oi.Quantity * p.Discounted_Price) AS TotalRevenue,
        ROUND(AVG(oi.Quantity * p.Discounted_Price),2) AS AvgRevenuePerOrder,
        COUNT(DISTINCT p.Product_ID) AS NumProductsSupplied
    FROM
        vendors v
        JOIN orders o ON o.VendorID = v.VendorID
        JOIN order_items oi ON oi.OrderID = o.OrderID
        JOIN products p ON p.Product_ID = oi.ProductID
    GROUP BY
        v.VendorID, v.VendorName
),
VendorRatings AS (
    SELECT
        v.VendorID,
        ROUND(AVG(p.Product_Rating),2) AS AvgProductRating
    FROM
        vendors v
        JOIN orders o ON o.VendorID = v.VendorID
        JOIN order_items oi ON oi.OrderID = o.OrderID
        JOIN products p ON p.Product_ID = oi.ProductID
    GROUP BY
        v.VendorID
),
VendorAnalysis AS (
    SELECT
        vt.VendorID,
        vt.VendorName,
        vt.TotalOrders,
        vt.TotalRevenue,
        vt.AvgRevenuePerOrder,
        vt.NumProductsSupplied,
        vr.AvgProductRating
    FROM
        VendorTotals vt
        LEFT JOIN VendorRatings vr ON vt.VendorID = vr.VendorID
)
SELECT
    VendorID,
    VendorName,
    TotalOrders,
    TotalRevenue,
    AvgRevenuePerOrder,
    NumProductsSupplied,
    COALESCE(AvgProductRating, 0) AS AvgProductRating
FROM
    VendorAnalysis
ORDER BY
    TotalRevenue DESC;


In [0]:
# Load tables
vendors = spark.table("vendors")
orders = spark.table("orders")
order_items = spark.table("order_items")
products = spark.table("products")

# Compute VendorTotals
vendor_totals = (
    vendors
    .join(orders, "VendorID")
    .join(order_items, "OrderID")
    .join(products, order_items["ProductID"] == products["Product_ID"])
    .groupBy("VendorID", "VendorName")
    .agg(
        F.countDistinct("orders.OrderID").alias("TotalOrders"),
        F.sum(order_items["Quantity"] * products["Discounted_Price"]).alias("TotalRevenue"),
        F.round(F.avg(order_items["Quantity"] * products["Discounted_Price"]), 2).alias("AvgRevenuePerOrder"),
        F.countDistinct(products["Product_ID"]).alias("NumProductsSupplied")
    )
)

# Compute VendorRatings
vendor_ratings = (
    vendors
    .join(orders, "VendorID")
    .join(order_items, "OrderID")
    .join(products, order_items["ProductID"] == products["Product_ID"])
    .groupBy("VendorID")
    .agg(
        F.round(F.avg(products["Product_Rating"]), 2).alias("AvgProductRating")
    )
)

# Compute VendorAnalysis
vendor_analysis = (
    vendor_totals
    .join(vendor_ratings, "VendorID", "left")
    .select(
        "VendorID",
        "VendorName",
        "TotalOrders",
        "TotalRevenue",
        "AvgRevenuePerOrder",
        "NumProductsSupplied",
        F.coalesce("AvgProductRating", F.lit(0)).alias("AvgProductRating")
    )
    .orderBy(F.col("TotalRevenue").desc())
)

# Display the final result
display(vendor_analysis)

#save to table
vendor_analysis.write.format("delta").mode("overwrite").saveAsTable("gold.vendor_analysis")

In [0]:
%sql
select * from gold.vendor_analysis